# 4.8 Grouping and Aggregating

## Contents:
### Set import variable
### Import orders_products_w_busy_days_times.pkl
### Define aggregation for entire df and analysis
### Add new field: Max order
### Add new field: Create loyalty flag per max_order data
### Add new field: Average price of items purchased as avg_spend¶
### Add new field: Create spending_flag per avg_spend data
### Add new field: Median frequency of customers as frequency_median
### Add new field: Create a frequency flag per the frequency_median
### Export final df
# -----------------------------------

### Import libraries and set import path

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Set path for use in import
path = r'C:\Users\XLT2\CFProjects\2023-04-07 Instacart Basket Analysis\02 Data'

### Import current data set

In [3]:
# Import orders_products_w_busy_days_times.pkl
df_ords_prods_merge = pd.read_pickle(os.path.join(path, 'Prepared Data', 'orders_products_w_busy_days_times.pkl'))

In [4]:
# Check data import
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days_2,busiest_hours
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,11,3.0,5,0,both,Mid-range product,Regularly busy,Regularly busy days,Busiest hours
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,17,20.0,1,1,both,Mid-range product,Regularly busy,Regularly busy days,Regularly busy hours
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,21,6.0,20,0,both,Mid-range product,Busiest day,Busiest days,Regularly busy hours
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,3,13,0.0,10,0,both,Mid-range product,Regularly busy,Least busy days,Regularly busy hours
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,17,9.0,11,1,both,Mid-range product,Least busy,Least busy days,Regularly busy hours


In [5]:
# Confirm rows
df_ords_prods_merge.shape

(32404859, 19)

### Step 2 - Define aggregation for entire df

In [6]:
# Groupby dept_id - aggregate order number by mean
df_ords_prods_merge.groupby('department_id').agg({'order_number' : ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### Step 3 - Analysis of aggregation results
The results are different, but similar. Since we arbitrarily sliced the first 1M records, we will lose records, but the trend holds relatively true. If we ran the sliced data repeatedly and added a larger slice of the data on each run, we would slowly move toward the results for our entire table.
This could be considered roughly similar to our lesson on a standard distribution. As we continue to add values, the bell curve distribution becomes more and more evident.

### Step 4 - Add new field: Max order

In [7]:
# Create new column, assign max order quantity for each user
df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [8]:
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days_2,busiest_hours,max_order
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,11,3.0,5,0,both,Mid-range product,Regularly busy,Regularly busy days,Busiest hours,32
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,17,20.0,1,1,both,Mid-range product,Regularly busy,Regularly busy days,Regularly busy hours,32
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,21,6.0,20,0,both,Mid-range product,Busiest day,Busiest days,Regularly busy hours,5
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,3,13,0.0,10,0,both,Mid-range product,Regularly busy,Least busy days,Regularly busy hours,3
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,17,9.0,11,1,both,Mid-range product,Least busy,Least busy days,Regularly busy hours,3


### Step 5 - Create loyalty flag per max_order data

In [9]:
# Create loyalty flag
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [10]:
df_ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [11]:
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,...,days_since_prior_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days_2,busiest_hours,max_order,loyalty_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,...,3.0,5,0,both,Mid-range product,Regularly busy,Regularly busy days,Busiest hours,32,Regular customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,...,20.0,1,1,both,Mid-range product,Regularly busy,Regularly busy days,Regularly busy hours,32,Regular customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,...,6.0,20,0,both,Mid-range product,Busiest day,Busiest days,Regularly busy hours,5,New customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,3,...,0.0,10,0,both,Mid-range product,Regularly busy,Least busy days,Regularly busy hours,3,New customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,...,9.0,11,1,both,Mid-range product,Least busy,Least busy days,Regularly busy hours,3,New customer


### Confirm loyalty_flag applied correctly

In [12]:
df_ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(60)

,user_id,loyalty_flag,order_number
0,138,Regular customer,28
1,138,Regular customer,30
2,709,New customer,2
3,764,New customer,1
4,764,New customer,3
5,777,Regular customer,16
6,825,New customer,3
7,910,Regular customer,12
8,1052,Regular customer,10
9,1052,Regular customer,15


### Step 6 - Add new field: Average price of items purchased as avg_spend

In [13]:
# Create new column, assign mean spend value for each user
df_ords_prods_merge['avg_spend'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [14]:
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,...,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days_2,busiest_hours,max_order,loyalty_flag,avg_spend
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,...,5,0,both,Mid-range product,Regularly busy,Regularly busy days,Busiest hours,32,Regular customer,6.935811
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,...,1,1,both,Mid-range product,Regularly busy,Regularly busy days,Regularly busy hours,32,Regular customer,6.935811
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,...,20,0,both,Mid-range product,Busiest day,Busiest days,Regularly busy hours,5,New customer,7.930208
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,3,...,10,0,both,Mid-range product,Regularly busy,Least busy days,Regularly busy hours,3,New customer,4.972414
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,...,11,1,both,Mid-range product,Least busy,Least busy days,Regularly busy hours,3,New customer,4.972414


### Create spending_flag per avg_spend data

In [15]:
# Create spending flag and assign a designation
df_ords_prods_merge.loc[df_ords_prods_merge['avg_spend'] >= 10, 'spending_flag'] = 'High spender'
df_ords_prods_merge.loc[df_ords_prods_merge['avg_spend'] < 10, 'spending_flag'] = 'Low spender'

In [16]:
# Change settings to remove limit on rows displayed
pd.options.display.max_rows = None

In [17]:
df_ords_prods_merge.head(100)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,...,reordered,_merge,price_range_loc,busiest_day,busiest_days_2,busiest_hours,max_order,loyalty_flag,avg_spend,spending_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,...,0,both,Mid-range product,Regularly busy,Regularly busy days,Busiest hours,32,Regular customer,6.935811,Low spender
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,...,1,both,Mid-range product,Regularly busy,Regularly busy days,Regularly busy hours,32,Regular customer,6.935811,Low spender
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,...,0,both,Mid-range product,Busiest day,Busiest days,Regularly busy hours,5,New customer,7.930208,Low spender
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,3,...,0,both,Mid-range product,Regularly busy,Least busy days,Regularly busy hours,3,New customer,4.972414,Low spender
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,...,1,both,Mid-range product,Least busy,Least busy days,Regularly busy hours,3,New customer,4.972414,Low spender
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,prior,16,1,...,0,both,Mid-range product,Regularly busy,Busiest days,Regularly busy hours,26,Regular customer,6.935398,Low spender
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,prior,3,2,...,0,both,Mid-range product,Regularly busy,Regularly busy days,Busiest hours,9,New customer,5.957576,Low spender
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,prior,12,3,...,0,both,Mid-range product,Regularly busy,Least busy days,Busiest hours,12,Regular customer,6.680000,Low spender
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,prior,10,1,...,0,both,Mid-range product,Regularly busy,Busiest days,Regularly busy hours,20,Regular customer,7.162500,Low spender
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,prior,15,1,...,1,both,Mid-range product,Regularly busy,Busiest days,Regularly busy hours,20,Regular customer,7.162500,Low spender


### Step 7 - Add new field: Median frequency of customers as frequency_median

In [19]:
# Create new column, assign median frequency value for each user
df_ords_prods_merge['frequency_median'] = df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [20]:
# Confirm write of field
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,...,_merge,price_range_loc,busiest_day,busiest_days_2,busiest_hours,max_order,loyalty_flag,avg_spend,spending_flag,frequency_median
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,...,both,Mid-range product,Regularly busy,Regularly busy days,Busiest hours,32,Regular customer,6.935811,Low spender,8.0
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,...,both,Mid-range product,Regularly busy,Regularly busy days,Regularly busy hours,32,Regular customer,6.935811,Low spender,8.0
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,...,both,Mid-range product,Busiest day,Busiest days,Regularly busy hours,5,New customer,7.930208,Low spender,6.0
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,3,...,both,Mid-range product,Regularly busy,Least busy days,Regularly busy hours,3,New customer,4.972414,Low spender,9.0
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,...,both,Mid-range product,Least busy,Least busy days,Regularly busy hours,3,New customer,4.972414,Low spender,9.0


### Create a frequency flag per the frequency_median

In [21]:
# Create frequency flag
df_ords_prods_merge.loc[df_ords_prods_merge['frequency_median'] > 20, 'frequency_flag'] = 'Non-frequent customer'
df_ords_prods_merge.loc[(df_ords_prods_merge['frequency_median'] <= 20) & (df_ords_prods_merge['frequency_median'] > 10), 'frequency_flag'] = 'Regular customer'
df_ords_prods_merge.loc[df_ords_prods_merge['frequency_median'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [22]:
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,...,price_range_loc,busiest_day,busiest_days_2,busiest_hours,max_order,loyalty_flag,avg_spend,spending_flag,frequency_median,frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,...,Mid-range product,Regularly busy,Regularly busy days,Busiest hours,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,...,Mid-range product,Regularly busy,Regularly busy days,Regularly busy hours,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,...,Mid-range product,Busiest day,Busiest days,Regularly busy hours,5,New customer,7.930208,Low spender,6.0,Frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,3,...,Mid-range product,Regularly busy,Least busy days,Regularly busy hours,3,New customer,4.972414,Low spender,9.0,Frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,...,Mid-range product,Least busy,Least busy days,Regularly busy hours,3,New customer,4.972414,Low spender,9.0,Frequent customer


### Export final data

In [23]:
# Export df_ords_prods_merge as orders_products_w_customer_flags.pkl
df_ords_prods_merge.to_pickle(os.path.join(path, 'Prepared Data', 'orders_products_w_customer_flags.pkl'))

In [24]:
# Import orders_products_w_customer_flags.pkl for good write confirmation
df_temp = pd.read_pickle(os.path.join(path, 'Prepared Data', 'orders_products_w_customer_flags.pkl'))

In [26]:
df_temp.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,...,price_range_loc,busiest_day,busiest_days_2,busiest_hours,max_order,loyalty_flag,avg_spend,spending_flag,frequency_median,frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,...,Mid-range product,Regularly busy,Regularly busy days,Busiest hours,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,...,Mid-range product,Regularly busy,Regularly busy days,Regularly busy hours,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,...,Mid-range product,Busiest day,Busiest days,Regularly busy hours,5,New customer,7.930208,Low spender,6.0,Frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,3,...,Mid-range product,Regularly busy,Least busy days,Regularly busy hours,3,New customer,4.972414,Low spender,9.0,Frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,...,Mid-range product,Least busy,Least busy days,Regularly busy hours,3,New customer,4.972414,Low spender,9.0,Frequent customer
